End to end floodplain mesh gen:
----------------------------------
```
Please use OCSMesh > version 1.6.4

Needs to prepare the following folders:
    path/
    |-- endtoend.py
    |-- inputs/
    |   |-- hgrid.gr3
    |   |-- highres_river_polys.shp
    |   |-- land_pearl_final.shp
    |   |-- dems/
    |       |-- *.tif  
    |-- outputs/
```
- **path:** your directory

- **hgrid.gr3:** STOFS3D mesh, used for defining the floodplain domain.
    Please download from: https://ccrm.vims.edu/yinglong/feiye/Public/OCSMesh/

- **highres_river_polys.shp:** Highly detailed RiverMapper polygons, used to create the river mesh.
    Please download from: https://ccrm.vims.edu/yinglong/feiye/Public/OCSMesh/
    For details on how to create this file, please refer to: https://github.com/schism-dev/RiverMeshTools

- **.tif files:** should be placed inside dems/, this is a list of all DEM you gathered from the different data sources.
    The other of the DEM's matter. Refer to the OCSMesh Technical Report for details

- **outputs/:** where outputs (final mesh) will be saved.


In [ ]:
import os
import time
from copy import deepcopy

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import (
                              Polygon, MultiPolygon, mapping
                             )
from shapely import intersection,difference

import ocsmesh

In [ ]:
path = r"/Your/Path/Here/"

## Floodplain Mesh:

Floodplain mesh, hfun. allows customization. Please see the OCSMesh manual: https://repository.library.noaa.gov/view/noaa/33879

In [ ]:
print("Begin Floodplain Mesh Gen")
start_time = time.time()

dem_paths = [f"{path}inputs/dems/{dem}" for dem in os.listdir(path+"inputs/dems/")]
geom_rast_list = [ocsmesh.Raster(f) for f in dem_paths if f[-4:] == '.tif']
hfun_rast_list = [ocsmesh.Raster(f) for f in dem_paths if f[-4:] == '.tif']

domain = gpd.read_file(path+"/land_pearl_final.shp")

#Mesh gen:
geom = ocsmesh.Geom(
    geom_rast_list,
    base_shape=domain.union_all(),
    base_shape_crs=domain.crs,
    # zmax=10
    )
hfun = ocsmesh.Hfun(
    hfun_rast_list,
    base_shape=domain.union_all(),
    base_shape_crs=geom.crs,
    hmin=50, hmax=500,
    method='fast')

hfun.add_constant_value(50, lower_bound=-20, upper_bound=1)
hfun.add_constant_value(100, lower_bound=1, upper_bound=5)
hfun.add_constant_value(200, lower_bound=5, upper_bound=10)

driver = ocsmesh.JigsawDriver(geom, hfun, crs=4326)
fp_mesh = driver.run()

hfun_mesh = ocsmesh.mesh.EuclideanMesh2D(hfun.msh_t())
ocsmesh.utils.reproject(mesh=hfun_mesh.msh_t,dst_crs=4326)
fp_mesh = ocsmesh.utils.fix_small_el(fp_mesh, hfun_mesh, u_limit= 1e-11, buffer_size= 0.001)
ocsmesh.utils.cleanup_isolates(fp_mesh)
ocsmesh.utils.cleanup_duplicates(fp_mesh)
ocsmesh.utils.put_id_tags(fp_mesh)

ocsmesh.Mesh(fp_mesh).write(path+"outputs/fp_mesh.2dm", format="2dm", overwrite=True)
del geom, hfun, geom_rast_list, hfun_rast_list, driver

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for FloodplainMesh: {elapsed_time} seconds")

## River Mesh:

Triangulates the RiverMapper polygons and clips it for the area of interest (floodplain domain)

In [ ]:
print("Begin River Mesh Gen")
start_time = time.time()

rm_poly = gpd.read_file(path+"inputs/highres_river_polys.shp")
river_tr = ocsmesh.utils.triangulate_rivermapper_poly(rm_poly)
river_tr = ocsmesh.utils.clip_mesh_by_shape(river_tr, domain.union_all(),adjacent_layers=12)
ocsmesh.Mesh(river_tr).write(path+"outputs/river_tr.2dm", format='2dm', overwrite=True)
del rm_poly, river_tr

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for RiverMesh: {elapsed_time} seconds")

## Floodplain+River Mesh for Pearl River Basin:

Creating the high resolution mesh

In [ ]:
print("Begin River Merge into the Floodplain")
start_time = time.time()

fp_mesh = ocsmesh.Mesh.open(path+"outputs/fp_mesh.2dm", crs=4326)
river_tr = ocsmesh.Mesh.open(path+"outputs/river_tr.2dm", crs=4326)

fp_r = ocsmesh.utils.merge_overlapping_meshes([fp_mesh.msh_t,river_tr.msh_t],adjacent_layers=2,buffer_size = 0.0005)

del fp_mesh, river_tr

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for Floodplain+River Mesh: {elapsed_time} seconds")

## High Resolution Mesh Merge into STOFS Mesh:

Relocating the High Resolution Mesh

In [ ]:
print("Begin Merging the High Resolution Mesh into the STOFS Mesh")
start_time = time.time()

STOFS_mesh = ocsmesh.Mesh.open(path+"inputs/hgrid.gr3", crs=4326)
fp_r_stofs = ocsmesh.utils.merge_overlapping_meshes([STOFS_mesh.msh_t, fp_r],adjacent_layers=2,buffer_size = 0.0005)

ocsmesh.Mesh(fp_r_stofs).write(path+"outputs/fp_r_stofs.2dm", format='2dm', overwrite=True)
ocsmesh.Mesh(fp_r_stofs).write(path+"outputs/fp_r_stofs.gr3", format='grd', overwrite=True)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for HighResolution+STOFS Mesh Merge: {elapsed_time} seconds")